# Retail Segmentation Excercise

**RFM** is a method used for analyzing customer value. It is commonly used in database marketing and direct marketing and has received particular attention in retail and professional services industries.

RFM stands for the three dimensions:

* **R**ecency – _How recently did the customer purchase?_
* **F**requency – _How often do they purchase?_
* **M**onetary Value – _How much do they spend?_

In [ ]:
!wget -q https://github.com/lexerdev/pairing-sessions/raw/main/data/orders.csv
!wget -q https://github.com/lexerdev/pairing-sessions/raw/main/data/customers.csv
!wget -q https://github.com/lexerdev/pairing-sessions/raw/main/data/order_items.csv

## Customer Data

This is a Brazilian ecommerce public dataset of orders made at [Olist Store](http://www.olist.com/). The dataset has information of 100k orders from 2016 to 2018 made at multiple marketplaces in Brazil.

Its features allows viewing an order from multiple dimensions: from **order status**, price, payment and freight performance to customer location, product attributes and finally reviews written by customers. We also released a geolocation dataset that relates Brazilian zip codes to lat/lng coordinates.

This is real commercial data, it has been anonymised.

**Entity Relationship Diagram**

Below is an Entity Relationship Diagram of the available data:

![ERD](https://github.com/lexerdev/pairing-sessions/raw/main/images/erd.png)


**Data Dictionary**

**customers**

* `customer_id`:  Customer ID number
* `customer_unique_id`: Unique ID of the Customer from resolution
* `customer_zip_code_prefix`: Zip code of customer
* `customer_city`: City of customer
* `customer_state`: State of customer

**orders**


* `order_id`: Order ID number
* `customer_id`:  Customer ID number
* `order_status`: Status of order process
* `order_purchase_timestamp`: Time of the product is ordered
* `order_approved_at`: Time of payment for the order
* `order_delivered_carrier_date`: Time of notifying the logistics in order process
* `order_delivered_customer_date`: Time of the product arrived at the consumer
* `order_estimated_delivery_date`: Estimated arrival time of the order

**order_items**

* `order_item_id`: The number of the product in the order (ex：If there are three items in the order, the order_item_id is 1, 2, 3)
* `product_id`: Product ID number
* `seller_id `: Seller ID number
* `shipping_limit_date`: The deadline for the seller deliver the goods to the logistics
* `price`: Product price
* `freight_value`: delivery fee

In [ ]:
import pandas as pd

orders = pd.read_csv(
    "orders.csv",
    parse_dates=[
        "order_purchase_timestamp",
        "order_approved_at",
        "order_delivered_carrier_date",
        "order_delivered_customer_date",
        "order_estimated_delivery_date",
    ],
)
customers = pd.read_csv("customers.csv")
geolocation = pd.read_csv("geolocation.csv")
order_items = pd.read_csv("order_items.csv")
order_payments = pd.read_csv("order_payments.csv")
order_reviews = pd.read_csv(
    "order_reviews.csv", parse_dates=["review_creation_date", "review_answer_timestamp"]
)
products = pd.read_csv("products.csv")
sellers = pd.read_csv("sellers.csv")

In [ ]:
customers.head()

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP


In [ ]:
orders.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26


In [ ]:
order_items.head()

,order_id,order_item_id,product_id,seller_id,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,58.90,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,239.90,19.93
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,199.00,17.87
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,12.99,12.79
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,199.90,18.14


## RFM Features


Let's see if we can setup a feature table like this:

| customer_unique_id               | recency | frequency | montetary |
|----------------------------------|---------|-----------|-----------|
| 9ef432eb6251297304e76186b10a928d | 7       | 0.8       | 6         |
| ...                              | ...     | ...       | ...       |

Where:
* `recency` is the count of months since the customer made a purchase
* `frequency` is the count of purchases of the customer in their activity period (e.g. 3 purchases / 2 months)
* `monetary` is a discretisation of `total_spend` in ten buckets, using [pd.qcut](https://pandas.pydata.org/docs/reference/api/pandas.qcut.html) is suitable
